In [1]:
import re
import operator
import networkx as nx

In [2]:
with open('../data/2024/day24.txt') as f:
    data = f.read()

In [3]:
wires, gates = data.split("\n\n")
wires = [tuple(w.split(': ')) for w in wires.splitlines()]
gates = [re.findall(r'(.*) (XOR|OR|AND) (.*) -> (.*)', g)[0] for g in gates.splitlines()]

In [4]:
G = nx.DiGraph()

# Initial wires
for w, v in wires:
    G.add_node(w)

# Add a pair of directed input edges to gates with one output
for i, (u, op, v, out) in enumerate(gates):
    gate_name = f'{op}{i}'
    G.add_node(gate_name, op=op)
    G.add_edge(u, gate_name)
    G.add_edge(v, gate_name)
    G.add_edge(gate_name, out)

# Outputs sorted in reverse order for bits
outputs = sorted(filter(lambda node: str(node).startswith('z'), G.nodes), reverse=True)

In [5]:
# Each gate has two inputs, though either can also be a gate
def get_gate_value(gate):
    inputs = G.predecessors(gate)
    a = get_node_value(next(inputs))
    b = get_node_value(next(inputs))

    if gate.startswith('XOR'):
        return operator.xor(a, b)
    elif gate.startswith('OR'):
        return operator.or_(a, b)
    elif gate.startswith('AND'):
        return operator.and_(a, b)
    else:
        raise ValueError('Unknown gate')

# Recursively determine the voltage of a node by looking at its inputs
def get_node_value(node):
    # This is a gate
    if G.nodes[node].get('op') in ['OR', 'AND', 'XOR']:
        return get_gate_value(node)
    else:
        # This isn't an initial input
        if list(G.predecessors(node)):
            return get_node_value(next(G.predecessors(node)))
        else: # This is an initial input
            return G.nodes[node].get('voltage', False)

In [6]:
# Part 1
# Add voltages to the initial wires as bools
for w, v in wires: G.nodes[w]['voltage'] = True if v == '1' else False

# Start at the outputs and recurse backwards to determine voltages
bits = ''.join(map(str,[1 if bit else 0 for bit in [get_node_value(output) for output in outputs]]))
print(f"Part 1: {int(bits, 2)}")

Part 1: 52038112429798


In [7]:
# Part 2
# Make collections for the x and y inputs
x_operand = sorted(filter(lambda node: str(node).startswith('x'), G.nodes), reverse=True)
y_operand = sorted(filter(lambda node: str(node).startswith('y'), G.nodes), reverse=True)

In [8]:
# Reset the input bits
for x in x_operand: G.nodes[x]['voltage'] = False
for y in y_operand: G.nodes[y]['voltage'] = False

# Add 0 in the x bank to individual bits in the y bank
for n in range(45):
    for y in y_operand:
        G.nodes[y]['voltage'] = False

    G.nodes[f'y{n:02}']['voltage'] = True

    # Calculate to show the errors
    bits = ''.join(map(str,[1 if bit else 0 for bit in [get_node_value(output) for output in outputs]]))

    if int(bits, 2) != pow(2, n):
        print(f"Answer: 2^{n} {int(bits, 2)} != {pow(2,n)}")

Answer: 2^12 8192 != 4096
Answer: 2^16 131072 != 65536
Answer: 2^24 33554432 != 16777216
Answer: 2^29 1073741824 != 536870912


In [9]:
# Output a graphviz DOT SVG and inspect the known bad
nx.nx_pydot.write_dot(G, 'visualizations/day24.dot')

In [10]:
# Fix bad wiring
bad_wires = "cph,jqn,kwb,qkf,tgr,z12,z16,z24"

G.remove_edge('XOR5', 'kwb')
G.add_edge('XOR5', 'z12')
G.remove_edge('AND163', 'z12')
G.add_edge('AND163', 'kwb')

G.remove_edge('XOR112', 'qkf')
G.add_edge('XOR112', 'z16')
G.remove_edge('AND195', 'z16')
G.add_edge('AND195', 'qkf')

G.remove_edge('OR50', 'z24')
G.add_edge('OR50', 'tgr')
G.remove_edge('XOR198', 'tgr')
G.add_edge('XOR198', 'z24')

G.remove_edge('XOR87', 'cph')
G.add_edge('XOR87', 'jqn')
G.remove_edge('AND46', 'jqn')
G.add_edge('AND46', 'cph')

In [11]:
# Reset the input bits
for x in x_operand: G.nodes[x]['voltage'] = False
for y in y_operand: G.nodes[y]['voltage'] = False

# Verify the bits work correctly
for n in range(45):
    for y in y_operand:
        G.nodes[y]['voltage'] = False

    G.nodes[f'y{n:02}']['voltage'] = True

    # Calculate
    bits = ''.join(map(str,[1 if bit else 0 for bit in [get_node_value(output) for output in outputs]]))
    print(f"Answer: {n} ^ {int(bits, 2)} == {pow(2,n)}")

Answer: 0 ^ 1 == 1
Answer: 1 ^ 2 == 2
Answer: 2 ^ 4 == 4
Answer: 3 ^ 8 == 8
Answer: 4 ^ 16 == 16
Answer: 5 ^ 32 == 32
Answer: 6 ^ 64 == 64
Answer: 7 ^ 128 == 128
Answer: 8 ^ 256 == 256
Answer: 9 ^ 512 == 512
Answer: 10 ^ 1024 == 1024
Answer: 11 ^ 2048 == 2048
Answer: 12 ^ 4096 == 4096
Answer: 13 ^ 8192 == 8192
Answer: 14 ^ 16384 == 16384
Answer: 15 ^ 32768 == 32768
Answer: 16 ^ 65536 == 65536
Answer: 17 ^ 131072 == 131072
Answer: 18 ^ 262144 == 262144
Answer: 19 ^ 524288 == 524288
Answer: 20 ^ 1048576 == 1048576
Answer: 21 ^ 2097152 == 2097152
Answer: 22 ^ 4194304 == 4194304
Answer: 23 ^ 8388608 == 8388608
Answer: 24 ^ 16777216 == 16777216
Answer: 25 ^ 33554432 == 33554432
Answer: 26 ^ 67108864 == 67108864
Answer: 27 ^ 134217728 == 134217728
Answer: 28 ^ 268435456 == 268435456
Answer: 29 ^ 536870912 == 536870912
Answer: 30 ^ 1073741824 == 1073741824
Answer: 31 ^ 2147483648 == 2147483648
Answer: 32 ^ 4294967296 == 4294967296
Answer: 33 ^ 8589934592 == 8589934592
Answer: 34 ^ 17179869184

In [12]:
print("Part 2:", bad_wires)

Part 2: cph,jqn,kwb,qkf,tgr,z12,z16,z24
